In [1]:
import tempfile

import pandas as pd
import csv
import numpy as np
import tensorflow as tf


/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#Load data and reduce the number of features for the baseline models

train = pd.read_csv('dataset/train.csv', dtype={'source_system_tab': str})
test = pd.read_csv('dataset/test.csv', dtype={'source_system_tab': str })
members = pd.read_csv('dataset/members.csv', dtype={'msno': str, 'city': str, 'registered_via': str})
songs = pd.read_csv('dataset/songs.csv', dtype={'genre_ids': str, 'language': str, 'song_length': int})

songs.drop(['composer', 'lyricist'], axis=1, inplace=True)

#Infer a missing value based on other features

songs.loc[605127, 'language'] = '31.0'

#Impute missing values

train.fillna(value='unknown', axis=1, inplace=True)
test.fillna(value='unknown', axis=1, inplace=True)
members.fillna(value='unknown', axis=1, inplace=True)
songs.fillna(value='unknown', axis=1, inplace=True)

#Convert feature columns related to member registration to datetime format

members['expiration_date'] = pd.to_datetime(members['expiration_date'], format='%Y%m%d')
members['registration_init_time'] = pd.to_datetime(members['registration_init_time'], format='%Y%m%d')

#Create a new feature indicating the number of days a member was registered

members['reg_duration'] = (members['expiration_date'] - members['registration_init_time']).dt.days


In [3]:
#Create lists of genre ids in the genre id column

#genres = songs['genre_ids'].str.split('|')

#Create a dataframe that stores genre IDs across multiple columns (one genre per column)

#genres = genres.apply(pd.Series).add_prefix('genre_')
#genres.to_csv('dataset/genres.csv', index=False)

genres = pd.read_csv('dataset/genres.csv', dtype=str)


In [4]:
#Merge the training and test data with song and member data

train_set = train.merge(songs, on='song_id')
train_set = train_set.merge(members, on='msno')
test_set = test.merge(songs, on='song_id', how='left')
test_set = test_set.merge(members, on='msno', how='left')

#Separate the submission ids from the test set

ids = test_set['id']
test_set.drop('id', axis=1, inplace=True)

#Impute missing values in merged training and test sets

train_set.fillna(value='unknown', axis=1, inplace=True)
test_set.fillna(value='unknown', axis=1, inplace=True)

#Impute missing song lengths with an integer value to avoid errors due to conflicting data types

test_set['song_length'] = test_set['song_length'].replace('unknown', -99)


In [5]:
#Shuffle the data and split off 20% of the training set for use as a validation set

split_ratio = 0.8

train_set = train_set.sample(frac=1, random_state=6)
val_set = train_set[int(split_ratio*train_set.shape[0]):]
train_set = train_set[:int(split_ratio*train_set.shape[0])]

#Separate the labels from the training and validation sets

y_train = train_set['target']
train_set.drop('target', axis=1, inplace=True)

y_val = val_set['target']
val_set.drop('target', axis=1, inplace=True)


In [6]:
#Designate the target feature name and the features to be used in the dataset

FEATURES = ['msno', 'gender', 'city', 'bd', 'registered_via',
            'song_id', 'artist_name', 'song_length', 'language', 'genre_ids',
            'source_system_tab', 'source_screen_name', 'source_type', 'reg_duration']

LABEL = 'target'

#Use the feature_column module to input each feature column into the model

target = tf.feature_column.categorical_column_with_identity(key='target', num_buckets=2)

length = tf.feature_column.numeric_column(key='song_length',
                                          default_value=-1,
                                          dtype=tf.int32)

duration = tf.feature_column.numeric_column(key='reg_duration',
                                            default_value=-1,
                                            dtype=tf.int32)

registered = tf.feature_column.categorical_column_with_vocabulary_list(key='registered_via',
                                                                       vocabulary_list=['7', '4', '9', '3', '13', '16'],
                                                                       dtype=tf.string,
                                                                       default_value=-99)

gender = tf.feature_column.categorical_column_with_vocabulary_list(key='gender',
                                                                   vocabulary_list=('female', 'male', 'unknown'),
                                                                   dtype=tf.string,
                                                                   default_value=-99)

city = tf.feature_column.categorical_column_with_vocabulary_list(key='city',
                                                          vocabulary_list=members['city'].unique(),
                                                          dtype=tf.string,
                                                          default_value=-99)

language = tf.feature_column.categorical_column_with_vocabulary_list(key='language',
                                                                     vocabulary_list=songs['language'].unique(),
                                                                     dtype=tf.string,
                                                                     default_value=-99)

artist = tf.feature_column.categorical_column_with_vocabulary_list(key='artist_name',
                                                                   vocabulary_list=songs['artist_name'].unique(),
                                                                   dtype=tf.string,
                                                                   default_value=-99)

tab = tf.feature_column.categorical_column_with_vocabulary_list(key='source_system_tab',
                                                                vocabulary_list=train['source_system_tab'].unique(),
                                                                dtype=tf.string,
                                                                default_value=-99)

screen = tf.feature_column.categorical_column_with_vocabulary_list(key='source_screen_name',
                                                                   vocabulary_list=train['source_screen_name'].unique(),
                                                                   dtype=tf.string,
                                                                   default_value=-99)

source = tf.feature_column.categorical_column_with_vocabulary_list(key='source_type',
                                                                   vocabulary_list=train['source_type'].unique(),
                                                                   dtype=tf.string,
                                                                   default_value=-99)

#Bucket categorical features with many unique categories using a hash table with a size of approximately (n/0.8)*2

msno = tf.feature_column.categorical_column_with_hash_bucket(key='msno',
                                                               hash_bucket_size=90000,
                                                               dtype=tf.string)

song_id = tf.feature_column.categorical_column_with_hash_bucket(key='song_id',
                                                             hash_bucket_size=6000000,
                                                             dtype=tf.string)

genre = tf.feature_column.categorical_column_with_vocabulary_list(key='genre_ids',
                                                                  vocabulary_list=genres['genre_0'].unique(),
                                                                  dtype=tf.string,
                                                                  default_value=-99)

hashed_genre = tf.feature_column.categorical_column_with_hash_bucket(key='genre_ids',
                                                                     hash_bucket_size=3000,
                                                                     dtype=tf.string)

#Perform one hot encoding on categorical features with few unique values

indicator_registered = tf.feature_column.indicator_column(registered)
indicator_gender = tf.feature_column.indicator_column(gender)
indicator_city = tf.feature_column.indicator_column(city)
indicator_genre = tf.feature_column.indicator_column(genre)
indicator_language = tf.feature_column.indicator_column(language)
indicator_tab = tf.feature_column.indicator_column(tab)
indicator_screen = tf.feature_column.indicator_column(screen)
indicator_source = tf.feature_column.indicator_column(source)

#Embed the categorical feature with <100 unique categories into dense vectors with approximately log2(n) dimensions

embedded_genre = tf.feature_column.embedding_column(genre, dimension=10)
embedded_song = tf.feature_column.embedding_column(song_id, dimension=22)
embedded_msno = tf.feature_column.embedding_column(msno, dimension=15)
embedded_artist = tf.feature_column.embedding_column(artist, dimension=15)

#Bucket member age into age ranges, with nonsensical values going into the 0-14 or the >80 buckets

age = tf.feature_column.numeric_column(key='bd',
                                       default_value=0,
                                       dtype=tf.int32)

age_bucket = tf.feature_column.bucketized_column(age, boundaries=[0, 14, 20, 30, 40, 50, 80])

#Assign features to be used in either the wide or the deep model (or both)

wide_columns = []
cross_columns = []
deep_columns = [
                indicator_gender, indicator_city, indicator_language, indicator_tab,
                indicator_screen, indicator_source, indicator_registered,
                embedded_msno, embedded_song, embedded_genre,
                #embedded_artist,
                duration,
                #length, age_bucket
                ]


In [7]:
def build_estimator(model_dir, model_type):
    if model_type == 'wide':
        model = tf.estimator.LinearClassifier(model_dir=model_dir,
                                              feature_columns=wide_columns + cross_columns)

    elif model_type == 'deep':
        model = tf.estimator.DNNClassifier(model_dir=model_dir,
                                           feature_columns=deep_columns,
                                           hidden_units=[1024, 512, 256],
                                           optimizer=tf.train.AdamOptimizer(learning_rate=0.001,
                                                                            name='Adam'))

    elif model_type == 'combined':
        model = tf.estimator.DNNLinearCombinedClassifier(model_dir=model_dir,
                                                         linear_feature_columns=cross_columns,
                                                         dnn_feature_columns=deep_columns,
                                                         dnn_hidden_units=[100, 50])

    return model


In [8]:
def input_fn(X, y, mode, batch_size):
    X.fillna(value='unknown', axis=1, inplace=True)    

    if mode == 'train':
        return tf.estimator.inputs.pandas_input_fn(x=pd.DataFrame({k: X[k].values for k in FEATURES}),
                                                    y=pd.Series(y.values),
                                                    batch_size=batch_size,
                                                    num_epochs=None,
                                                    shuffle=True,
                                                    num_threads=8,
                                                    target_column='target')
    
    elif mode == 'eval':
        return tf.estimator.inputs.pandas_input_fn(x = pd.DataFrame({k: X[k].values for k in FEATURES}),
                                                    y = pd.Series(y.values),
                                                    batch_size=batch_size,
                                                    num_epochs=1,
                                                    shuffle=False,
                                                    num_threads=1,
                                                    target_column='target')
    
    elif mode == 'predict':
        return tf.estimator.inputs.pandas_input_fn(x=pd.DataFrame({k: X[k].values for k in FEATURES}),
                                                    batch_size=batch_size,
                                                    num_epochs=1,
                                                    shuffle=False,num_threads=1)
    

In [9]:
def train_model(model_dir, model_type, train_steps, X_train, y_train, X_test, y_test, batch_size):

#Create a temporary directory to store the model if no model directory argument is given

    model_dir = tempfile.mkdtemp() if not model_dir else model_dir
    
    print('build_estimator')
    model = build_estimator(model_dir, model_type)
    
    print('train start')
    
    model.train(input_fn=input_fn(X_train, y_train, mode='train', batch_size=batch_size),
                max_steps=train_steps)
    
#Evaluate the trained model on a separate validation set in n/batch_size steps
    
    model.evaluate(input_fn=input_fn(X_test, y_test, mode='eval', batch_size=batch_size),
                        steps=(X_test.shape[0]//batch_size + 1))

    print('end!')
    
    return model
    

In [10]:
deep_model = train_model(model_dir='model/', model_type='deep', train_steps=600000,
                         X_train=train_set, y_train=y_train,
                         X_test=val_set, y_test=y_val,
                         batch_size=100)


build_estimator
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efc411d5e80>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
train start
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from model/model.ckpt-134774
INFO:tensorflow:Saving checkpoints for 134775 into model/model.ckpt.
INFO:tensorflow:loss = 60.6761, step = 134775
INFO:tensorflow:global_step/sec: 11.5798
INFO:tensorflow:loss = 61.5613, step = 134875 (8.637 sec)
INFO:tensorflow:global_step/sec: 11.8455
INFO:tensorflow:loss = 63.3173, step =

INFO:tensorflow:global_step/sec: 11.8391
INFO:tensorflow:loss = 62.594, step = 142275 (8.447 sec)
INFO:tensorflow:global_step/sec: 11.8402
INFO:tensorflow:loss = 67.2163, step = 142375 (8.446 sec)
INFO:tensorflow:global_step/sec: 11.8556
INFO:tensorflow:loss = 57.1498, step = 142475 (8.435 sec)
INFO:tensorflow:global_step/sec: 11.8498
INFO:tensorflow:loss = 58.7249, step = 142575 (8.440 sec)
INFO:tensorflow:global_step/sec: 11.8042
INFO:tensorflow:loss = 62.1003, step = 142675 (8.470 sec)
INFO:tensorflow:global_step/sec: 11.8371
INFO:tensorflow:loss = 60.8692, step = 142775 (8.448 sec)
INFO:tensorflow:global_step/sec: 11.8571
INFO:tensorflow:loss = 59.2135, step = 142875 (8.434 sec)
INFO:tensorflow:global_step/sec: 11.8614
INFO:tensorflow:loss = 50.0386, step = 142975 (8.430 sec)
INFO:tensorflow:global_step/sec: 11.8531
INFO:tensorflow:loss = 52.7744, step = 143075 (8.437 sec)
INFO:tensorflow:global_step/sec: 11.8716
INFO:tensorflow:loss = 60.4192, step = 143175 (8.423 sec)
INFO:tensor

INFO:tensorflow:loss = 57.2972, step = 150475 (8.431 sec)
INFO:tensorflow:global_step/sec: 11.852
INFO:tensorflow:loss = 59.2792, step = 150575 (8.436 sec)
INFO:tensorflow:global_step/sec: 11.8288
INFO:tensorflow:loss = 54.6643, step = 150675 (8.454 sec)
INFO:tensorflow:global_step/sec: 11.8398
INFO:tensorflow:loss = 64.6967, step = 150775 (8.446 sec)
INFO:tensorflow:global_step/sec: 11.8639
INFO:tensorflow:loss = 56.3086, step = 150875 (8.429 sec)
INFO:tensorflow:global_step/sec: 11.8077
INFO:tensorflow:loss = 72.7597, step = 150975 (8.469 sec)
INFO:tensorflow:global_step/sec: 11.8558
INFO:tensorflow:loss = 69.2426, step = 151075 (8.436 sec)
INFO:tensorflow:global_step/sec: 11.8617
INFO:tensorflow:loss = 56.3706, step = 151175 (8.429 sec)
INFO:tensorflow:global_step/sec: 11.8528
INFO:tensorflow:loss = 56.4826, step = 151275 (8.437 sec)
INFO:tensorflow:global_step/sec: 11.8397
INFO:tensorflow:loss = 58.366, step = 151375 (8.446 sec)
INFO:tensorflow:global_step/sec: 11.8637
INFO:tensorf

INFO:tensorflow:global_step/sec: 11.8601
INFO:tensorflow:loss = 62.1552, step = 158775 (8.432 sec)
INFO:tensorflow:global_step/sec: 11.8619
INFO:tensorflow:loss = 58.7667, step = 158875 (8.430 sec)
INFO:tensorflow:global_step/sec: 11.8666
INFO:tensorflow:loss = 55.1774, step = 158975 (8.427 sec)
INFO:tensorflow:global_step/sec: 11.8512
INFO:tensorflow:loss = 63.0275, step = 159075 (8.438 sec)
INFO:tensorflow:global_step/sec: 11.8498
INFO:tensorflow:loss = 61.0474, step = 159175 (8.439 sec)
INFO:tensorflow:global_step/sec: 11.8064
INFO:tensorflow:loss = 65.2466, step = 159275 (8.470 sec)
INFO:tensorflow:global_step/sec: 11.8588
INFO:tensorflow:loss = 64.2966, step = 159375 (8.432 sec)
INFO:tensorflow:global_step/sec: 11.869
INFO:tensorflow:loss = 59.7561, step = 159475 (8.425 sec)
INFO:tensorflow:global_step/sec: 11.8611
INFO:tensorflow:loss = 61.466, step = 159575 (8.431 sec)
INFO:tensorflow:global_step/sec: 11.872
INFO:tensorflow:loss = 64.8824, step = 159675 (8.423 sec)
INFO:tensorfl

INFO:tensorflow:loss = 63.5632, step = 166975 (8.461 sec)
INFO:tensorflow:global_step/sec: 11.8451
INFO:tensorflow:loss = 57.4526, step = 167075 (8.442 sec)
INFO:tensorflow:global_step/sec: 11.8304
INFO:tensorflow:loss = 59.8574, step = 167175 (8.453 sec)
INFO:tensorflow:global_step/sec: 11.8394
INFO:tensorflow:loss = 60.1502, step = 167275 (8.447 sec)
INFO:tensorflow:global_step/sec: 11.8614
INFO:tensorflow:loss = 58.8778, step = 167375 (8.430 sec)
INFO:tensorflow:global_step/sec: 11.8237
INFO:tensorflow:loss = 64.3493, step = 167475 (8.458 sec)
INFO:tensorflow:global_step/sec: 11.8335
INFO:tensorflow:loss = 58.8241, step = 167575 (8.450 sec)
INFO:tensorflow:global_step/sec: 11.8459
INFO:tensorflow:loss = 66.9098, step = 167675 (8.442 sec)
INFO:tensorflow:global_step/sec: 11.8386
INFO:tensorflow:loss = 60.9367, step = 167775 (8.447 sec)
INFO:tensorflow:global_step/sec: 11.8609
INFO:tensorflow:loss = 54.7661, step = 167875 (8.431 sec)
INFO:tensorflow:global_step/sec: 11.8663
INFO:tenso

...

INFO:tensorflow:Evaluation [13952/14755]
INFO:tensorflow:Evaluation [13953/14755]
INFO:tensorflow:Evaluation [13954/14755]
INFO:tensorflow:Evaluation [13955/14755]
INFO:tensorflow:Evaluation [13956/14755]
INFO:tensorflow:Evaluation [13957/14755]
INFO:tensorflow:Evaluation [13958/14755]
INFO:tensorflow:Evaluation [13959/14755]
INFO:tensorflow:Evaluation [13960/14755]
INFO:tensorflow:Evaluation [13961/14755]
INFO:tensorflow:Evaluation [13962/14755]
INFO:tensorflow:Evaluation [13963/14755]
INFO:tensorflow:Evaluation [13964/14755]
INFO:tensorflow:Evaluation [13965/14755]
INFO:tensorflow:Evaluation [13966/14755]
INFO:tensorflow:Evaluation [13967/14755]
INFO:tensorflow:Evaluation [13968/14755]
INFO:tensorflow:Evaluation [13969/14755]
INFO:tensorflow:Evaluation [13970/14755]
INFO:tensorflow:Evaluation [13971/14755]
INFO:tensorflow:Evaluation [13972/14755]
INFO:tensorflow:Evaluation [13973/14755]
INFO:tensorflow:Evaluation [13974/14755]
INFO:tensorflow:Evaluation [13975/14755]
INFO:tensorflow:

INFO:tensorflow:Evaluation [14152/14755]
INFO:tensorflow:Evaluation [14153/14755]
INFO:tensorflow:Evaluation [14154/14755]
INFO:tensorflow:Evaluation [14155/14755]
INFO:tensorflow:Evaluation [14156/14755]
INFO:tensorflow:Evaluation [14157/14755]
INFO:tensorflow:Evaluation [14158/14755]
INFO:tensorflow:Evaluation [14159/14755]
INFO:tensorflow:Evaluation [14160/14755]
INFO:tensorflow:Evaluation [14161/14755]
INFO:tensorflow:Evaluation [14162/14755]
INFO:tensorflow:Evaluation [14163/14755]
INFO:tensorflow:Evaluation [14164/14755]
INFO:tensorflow:Evaluation [14165/14755]
INFO:tensorflow:Evaluation [14166/14755]
INFO:tensorflow:Evaluation [14167/14755]
INFO:tensorflow:Evaluation [14168/14755]
INFO:tensorflow:Evaluation [14169/14755]
INFO:tensorflow:Evaluation [14170/14755]
INFO:tensorflow:Evaluation [14171/14755]
INFO:tensorflow:Evaluation [14172/14755]
INFO:tensorflow:Evaluation [14173/14755]
INFO:tensorflow:Evaluation [14174/14755]
INFO:tensorflow:Evaluation [14175/14755]
INFO:tensorflow:

INFO:tensorflow:Evaluation [14351/14755]
INFO:tensorflow:Evaluation [14352/14755]
INFO:tensorflow:Evaluation [14353/14755]
INFO:tensorflow:Evaluation [14354/14755]
INFO:tensorflow:Evaluation [14355/14755]
INFO:tensorflow:Evaluation [14356/14755]
INFO:tensorflow:Evaluation [14357/14755]
INFO:tensorflow:Evaluation [14358/14755]
INFO:tensorflow:Evaluation [14359/14755]
INFO:tensorflow:Evaluation [14360/14755]
INFO:tensorflow:Evaluation [14361/14755]
INFO:tensorflow:Evaluation [14362/14755]
INFO:tensorflow:Evaluation [14363/14755]
INFO:tensorflow:Evaluation [14364/14755]
INFO:tensorflow:Evaluation [14365/14755]
INFO:tensorflow:Evaluation [14366/14755]
INFO:tensorflow:Evaluation [14367/14755]
INFO:tensorflow:Evaluation [14368/14755]
INFO:tensorflow:Evaluation [14369/14755]
INFO:tensorflow:Evaluation [14370/14755]
INFO:tensorflow:Evaluation [14371/14755]
INFO:tensorflow:Evaluation [14372/14755]
INFO:tensorflow:Evaluation [14373/14755]
INFO:tensorflow:Evaluation [14374/14755]
INFO:tensorflow:

INFO:tensorflow:Evaluation [14551/14755]
INFO:tensorflow:Evaluation [14552/14755]
INFO:tensorflow:Evaluation [14553/14755]
INFO:tensorflow:Evaluation [14554/14755]
INFO:tensorflow:Evaluation [14555/14755]
INFO:tensorflow:Evaluation [14556/14755]
INFO:tensorflow:Evaluation [14557/14755]
INFO:tensorflow:Evaluation [14558/14755]
INFO:tensorflow:Evaluation [14559/14755]
INFO:tensorflow:Evaluation [14560/14755]
INFO:tensorflow:Evaluation [14561/14755]
INFO:tensorflow:Evaluation [14562/14755]
INFO:tensorflow:Evaluation [14563/14755]
INFO:tensorflow:Evaluation [14564/14755]
INFO:tensorflow:Evaluation [14565/14755]
INFO:tensorflow:Evaluation [14566/14755]
INFO:tensorflow:Evaluation [14567/14755]
INFO:tensorflow:Evaluation [14568/14755]
INFO:tensorflow:Evaluation [14569/14755]
INFO:tensorflow:Evaluation [14570/14755]
INFO:tensorflow:Evaluation [14571/14755]
INFO:tensorflow:Evaluation [14572/14755]
INFO:tensorflow:Evaluation [14573/14755]
INFO:tensorflow:Evaluation [14574/14755]
INFO:tensorflow:

INFO:tensorflow:Evaluation [14751/14755]
INFO:tensorflow:Evaluation [14752/14755]
INFO:tensorflow:Evaluation [14753/14755]
INFO:tensorflow:Evaluation [14754/14755]
INFO:tensorflow:Evaluation [14755/14755]
INFO:tensorflow:Finished evaluation at 2017-11-25-14:43:43
INFO:tensorflow:Saving dict for global step 600000: accuracy = 0.702063, accuracy_baseline = 0.503512, auc = 0.771803, auc_precision_recall = 0.72545, average_loss = 0.578341, global_step = 600000, label/mean = 0.503512, loss = 57.8326, prediction/mean = 0.525922
end!


In [11]:
results = deep_model.evaluate(input_fn=input_fn(train_set, y_train, mode='eval', batch_size=1000),
                              steps=1476)


INFO:tensorflow:Starting evaluation at 2017-11-25-14:43:50
INFO:tensorflow:Restoring parameters from model/model.ckpt-600000
INFO:tensorflow:Evaluation [1/1476]
INFO:tensorflow:Evaluation [2/1476]
INFO:tensorflow:Evaluation [3/1476]
INFO:tensorflow:Evaluation [4/1476]
INFO:tensorflow:Evaluation [5/1476]
INFO:tensorflow:Evaluation [6/1476]
INFO:tensorflow:Evaluation [7/1476]
INFO:tensorflow:Evaluation [8/1476]
INFO:tensorflow:Evaluation [9/1476]
INFO:tensorflow:Evaluation [10/1476]
INFO:tensorflow:Evaluation [11/1476]
INFO:tensorflow:Evaluation [12/1476]
INFO:tensorflow:Evaluation [13/1476]
INFO:tensorflow:Evaluation [14/1476]
INFO:tensorflow:Evaluation [15/1476]
INFO:tensorflow:Evaluation [16/1476]
INFO:tensorflow:Evaluation [17/1476]
INFO:tensorflow:Evaluation [18/1476]
INFO:tensorflow:Evaluation [19/1476]
INFO:tensorflow:Evaluation [20/1476]
INFO:tensorflow:Evaluation [21/1476]
INFO:tensorflow:Evaluation [22/1476]
INFO:tensorflow:Evaluation [23/1476]
INFO:tensorflow:Evaluation [24/14

INFO:tensorflow:Evaluation [213/1476]
INFO:tensorflow:Evaluation [214/1476]
INFO:tensorflow:Evaluation [215/1476]
INFO:tensorflow:Evaluation [216/1476]
INFO:tensorflow:Evaluation [217/1476]
INFO:tensorflow:Evaluation [218/1476]
INFO:tensorflow:Evaluation [219/1476]
INFO:tensorflow:Evaluation [220/1476]
INFO:tensorflow:Evaluation [221/1476]
INFO:tensorflow:Evaluation [222/1476]
INFO:tensorflow:Evaluation [223/1476]
INFO:tensorflow:Evaluation [224/1476]
INFO:tensorflow:Evaluation [225/1476]
INFO:tensorflow:Evaluation [226/1476]
INFO:tensorflow:Evaluation [227/1476]
INFO:tensorflow:Evaluation [228/1476]
INFO:tensorflow:Evaluation [229/1476]
INFO:tensorflow:Evaluation [230/1476]
INFO:tensorflow:Evaluation [231/1476]
INFO:tensorflow:Evaluation [232/1476]
INFO:tensorflow:Evaluation [233/1476]
INFO:tensorflow:Evaluation [234/1476]
INFO:tensorflow:Evaluation [235/1476]
INFO:tensorflow:Evaluation [236/1476]
INFO:tensorflow:Evaluation [237/1476]
INFO:tensorflow:Evaluation [238/1476]
INFO:tensorf

INFO:tensorflow:Evaluation [429/1476]
INFO:tensorflow:Evaluation [430/1476]
INFO:tensorflow:Evaluation [431/1476]
INFO:tensorflow:Evaluation [432/1476]
INFO:tensorflow:Evaluation [433/1476]
INFO:tensorflow:Evaluation [434/1476]
INFO:tensorflow:Evaluation [435/1476]
INFO:tensorflow:Evaluation [436/1476]
INFO:tensorflow:Evaluation [437/1476]
INFO:tensorflow:Evaluation [438/1476]
INFO:tensorflow:Evaluation [439/1476]
INFO:tensorflow:Evaluation [440/1476]
INFO:tensorflow:Evaluation [441/1476]
INFO:tensorflow:Evaluation [442/1476]
INFO:tensorflow:Evaluation [443/1476]
INFO:tensorflow:Evaluation [444/1476]
INFO:tensorflow:Evaluation [445/1476]
INFO:tensorflow:Evaluation [446/1476]
INFO:tensorflow:Evaluation [447/1476]
INFO:tensorflow:Evaluation [448/1476]
INFO:tensorflow:Evaluation [449/1476]
INFO:tensorflow:Evaluation [450/1476]
INFO:tensorflow:Evaluation [451/1476]
INFO:tensorflow:Evaluation [452/1476]
INFO:tensorflow:Evaluation [453/1476]
INFO:tensorflow:Evaluation [454/1476]
INFO:tensorf

INFO:tensorflow:Evaluation [645/1476]
INFO:tensorflow:Evaluation [646/1476]
INFO:tensorflow:Evaluation [647/1476]
INFO:tensorflow:Evaluation [648/1476]
INFO:tensorflow:Evaluation [649/1476]
INFO:tensorflow:Evaluation [650/1476]
INFO:tensorflow:Evaluation [651/1476]
INFO:tensorflow:Evaluation [652/1476]
INFO:tensorflow:Evaluation [653/1476]
INFO:tensorflow:Evaluation [654/1476]
INFO:tensorflow:Evaluation [655/1476]
INFO:tensorflow:Evaluation [656/1476]
INFO:tensorflow:Evaluation [657/1476]
INFO:tensorflow:Evaluation [658/1476]
INFO:tensorflow:Evaluation [659/1476]
INFO:tensorflow:Evaluation [660/1476]
INFO:tensorflow:Evaluation [661/1476]
INFO:tensorflow:Evaluation [662/1476]
INFO:tensorflow:Evaluation [663/1476]
INFO:tensorflow:Evaluation [664/1476]
INFO:tensorflow:Evaluation [665/1476]
INFO:tensorflow:Evaluation [666/1476]
INFO:tensorflow:Evaluation [667/1476]
INFO:tensorflow:Evaluation [668/1476]
INFO:tensorflow:Evaluation [669/1476]
INFO:tensorflow:Evaluation [670/1476]
INFO:tensorf

INFO:tensorflow:Evaluation [861/1476]
INFO:tensorflow:Evaluation [862/1476]
INFO:tensorflow:Evaluation [863/1476]
INFO:tensorflow:Evaluation [864/1476]
INFO:tensorflow:Evaluation [865/1476]
INFO:tensorflow:Evaluation [866/1476]
INFO:tensorflow:Evaluation [867/1476]
INFO:tensorflow:Evaluation [868/1476]
INFO:tensorflow:Evaluation [869/1476]
INFO:tensorflow:Evaluation [870/1476]
INFO:tensorflow:Evaluation [871/1476]
INFO:tensorflow:Evaluation [872/1476]
INFO:tensorflow:Evaluation [873/1476]
INFO:tensorflow:Evaluation [874/1476]
INFO:tensorflow:Evaluation [875/1476]
INFO:tensorflow:Evaluation [876/1476]
INFO:tensorflow:Evaluation [877/1476]
INFO:tensorflow:Evaluation [878/1476]
INFO:tensorflow:Evaluation [879/1476]
INFO:tensorflow:Evaluation [880/1476]
INFO:tensorflow:Evaluation [881/1476]
INFO:tensorflow:Evaluation [882/1476]
INFO:tensorflow:Evaluation [883/1476]
INFO:tensorflow:Evaluation [884/1476]
INFO:tensorflow:Evaluation [885/1476]
INFO:tensorflow:Evaluation [886/1476]
INFO:tensorf

INFO:tensorflow:Evaluation [1075/1476]
INFO:tensorflow:Evaluation [1076/1476]
INFO:tensorflow:Evaluation [1077/1476]
INFO:tensorflow:Evaluation [1078/1476]
INFO:tensorflow:Evaluation [1079/1476]
INFO:tensorflow:Evaluation [1080/1476]
INFO:tensorflow:Evaluation [1081/1476]
INFO:tensorflow:Evaluation [1082/1476]
INFO:tensorflow:Evaluation [1083/1476]
INFO:tensorflow:Evaluation [1084/1476]
INFO:tensorflow:Evaluation [1085/1476]
INFO:tensorflow:Evaluation [1086/1476]
INFO:tensorflow:Evaluation [1087/1476]
INFO:tensorflow:Evaluation [1088/1476]
INFO:tensorflow:Evaluation [1089/1476]
INFO:tensorflow:Evaluation [1090/1476]
INFO:tensorflow:Evaluation [1091/1476]
INFO:tensorflow:Evaluation [1092/1476]
INFO:tensorflow:Evaluation [1093/1476]
INFO:tensorflow:Evaluation [1094/1476]
INFO:tensorflow:Evaluation [1095/1476]
INFO:tensorflow:Evaluation [1096/1476]
INFO:tensorflow:Evaluation [1097/1476]
INFO:tensorflow:Evaluation [1098/1476]
INFO:tensorflow:Evaluation [1099/1476]
INFO:tensorflow:Evaluatio

INFO:tensorflow:Evaluation [1286/1476]
INFO:tensorflow:Evaluation [1287/1476]
INFO:tensorflow:Evaluation [1288/1476]
INFO:tensorflow:Evaluation [1289/1476]
INFO:tensorflow:Evaluation [1290/1476]
INFO:tensorflow:Evaluation [1291/1476]
INFO:tensorflow:Evaluation [1292/1476]
INFO:tensorflow:Evaluation [1293/1476]
INFO:tensorflow:Evaluation [1294/1476]
INFO:tensorflow:Evaluation [1295/1476]
INFO:tensorflow:Evaluation [1296/1476]
INFO:tensorflow:Evaluation [1297/1476]
INFO:tensorflow:Evaluation [1298/1476]
INFO:tensorflow:Evaluation [1299/1476]
INFO:tensorflow:Evaluation [1300/1476]
INFO:tensorflow:Evaluation [1301/1476]
INFO:tensorflow:Evaluation [1302/1476]
INFO:tensorflow:Evaluation [1303/1476]
INFO:tensorflow:Evaluation [1304/1476]
INFO:tensorflow:Evaluation [1305/1476]
INFO:tensorflow:Evaluation [1306/1476]
INFO:tensorflow:Evaluation [1307/1476]
INFO:tensorflow:Evaluation [1308/1476]
INFO:tensorflow:Evaluation [1309/1476]
INFO:tensorflow:Evaluation [1310/1476]
INFO:tensorflow:Evaluatio

In [13]:
predictions = deep_model.predict(input_fn=input_fn(test_set, None, mode='predict',
                                                   batch_size=10000))

submission = list()

for row in predictions:
    submission.append(row['probabilities'][1])

pd.DataFrame(data={'id': ids,
                   'target': np.array(submission)}).to_csv('submissions/embedded_deep.csv',
                                                           header=['id', 'target'],
                                                           index=False)


INFO:tensorflow:Restoring parameters from model/model.ckpt-600000
